In [16]:
%apt-get install portaudio19-dev
%pip install SpeechRecognition
%pip install pyaudio
%pip install pydub
%pip install tflearn
%pip install tensorflow
%pip install nltk
%pip install numpy
%pip install scipy
%pip install gTTS

UsageError: Line magic function `%apt-get` not found.


In [ ]:
from gtts import gTTS
import os
import nltk
nltk.download('punkt')
import tflearn
import tensorflow as tf
import numpy as np
import random
import json
import speech_recognition as sr

from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

json_path = r"E:\first_aid\intents.json"  # Use 'r' before the path to handle backslashes
with open(json_path) as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)


model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

def listen_to_voice():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            return text
        except sr.WaitTimeoutError:
            print("Timeout: No speech detected.")
            return None
        except sr.UnknownValueError:
            print("Sorry, I couldn't understand the audio.")
            return None
        except sr.RequestError as e:
            print(f"Error while requesting the speech recognition service: {e}")
            return None



def chat():
    print("Start Talking with the bot (type quit to stop)!")
    while True:
        inp = listen_to_voice()
        if inp and inp.lower() == "quit":
            break

        if inp:
            results = model.predict(np.array([bag_of_words(inp, words)]))[0]
            results_index = np.argmax(results)
            tag = labels[results_index]

            if results[results_index] > 0.5:
                for tg in data["intents"]:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                response = random.choice(responses)
                print(response)

                if isinstance(response, (list, tuple)):
                    response = [item for sublist in response for item in sublist]

                text_to_speech(response)  # Convert response to speech
            else:
                print("I didn't get that, try again")
                
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    os.system("response.mp3")


if __name__ == "__main__":
    print("Bot: Hello! How can I assist you today?")
    while True:
        inp = listen_to_voice()
        if inp and inp.lower() == "quit":
            break

        if inp:
            results = model.predict([bag_of_words(inp, words)])[0]
            results_index = np.argmax(results)
            tag = labels[results_index]

            if results[results_index] > 0.5:
                for tg in data["intents"]:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                response = random.choice(responses)
                print("Bot:", response)
                text_to_speech(response)  # Convert response to speech
            else:
                print("Bot: I didn't get that, try again")

In [34]:
import torch
import torch.nn as nn
import speech_recognition as sr
from gtts import gTTS
import os
import random
import json

# Load your intents data here
json_path = r"E:\first_aid\intents.json"  # Use 'r' before the path to handle backslashes
with open(json_path) as file:
    data = json.load(file)

# Define your PyTorch-based chatbot model class here (similar to your previous code)
class YourChatbotModelClass(nn.Module):
    # Define your model architecture and methods here
    pass

# Load your trained PyTorch model here
model = YourChatbotModelClass()
model.load_state_dict(torch.load("your_model.pth"))
model.eval()

# Define your voice recognition function
def listen_to_voice():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            return text
        except sr.WaitTimeoutError:
            print("Timeout: No speech detected.")
            return None
        except sr.UnknownValueError:
            print("Sorry, I couldn't understand the audio.")
            return None
        except sr.RequestError as e:
            print(f"Error while requesting the speech recognition service: {e}")
            return None

# Define your text-to-speech function
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    os.system("response.mp3")

# Define your chat function
def chat():
    print("Bot: Hello! How can I assist you today?")
    while True:
        inp = listen_to_voice()
        if inp and inp.lower() == "quit":
            break

        if inp:
            results = model.predict(np.array([bag_of_words(inp, words)]))[0]
            results_index = np.argmax(results)
            tag = labels[results_index]

            if results[results_index] > 0.5:
                for tg in data["intents"]:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                response = random.choice(responses)
                print(response)

                if isinstance(response, (list, tuple)):
                    response = [item for sublist in response for item in sublist]

                text_to_speech(response)  # Convert response to speech
            else:
                print("I didn't get that, try again")


            # Replace this with your actual logic for processing and generating responses
# Call the generate_response_from_model function in the chat loop
if inp:
    response = generate_response_from_model(inp, model, data)
    print("Bot:", response)
    text_to_speech(response)  # Convert response to speech
  # Convert response to speech

# Call the chat function
if __name__ == "__main__":
    chat()


FileNotFoundError: [Errno 2] No such file or directory: 'your_model.pth'

In [36]:
import torch
import torch.nn as nn
import nltk
import random
import json
import pickle
import numpy as np
import speech_recognition as sr
from gtts import gTTS
import os

stemmer = nltk.stem.lancaster.LancasterStemmer()

# Load intents data
with open("intents.json") as file:
    data = json.load(file)

# Load preprocessed training data
with open("training_data.pkl", "rb") as file:
    words, labels, training, output = pickle.load(file)

# Define PyTorch chatbot model
class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatbotModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.rnn(embedded)
        return self.fc(output)

# Initialize the model
input_size = len(words)
hidden_size = 128
output_size = len(labels)
model = ChatbotModel(input_size, hidden_size, output_size)

# Load trained model parameters
model.load_state_dict(torch.load("chatbot_model.pth"))
model.eval()

# Define generate_response_from_model function
def generate_response_from_model(inp, model, words, labels, data):
    tokenized = nltk.word_tokenize(inp)
    stemmed = [stemmer.stem(word.lower()) for word in tokenized]
    bow = np.zeros(len(words))
    for idx, w in enumerate(words):
        if w in stemmed:
            bow[idx] = 1

    tensor_input = torch.tensor(bow, dtype=torch.float32).view(1, -1)
    with torch.no_grad():
        output = model(tensor_input)
        response_idx = torch.argmax(output).item()
        tag = labels[response_idx]

    for intent in data["intents"]:
        if intent["tag"] == tag:
            responses = intent["responses"]
            return random.choice(responses)

# Define voice recognition function
def listen_to_voice():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            return text
        except sr.WaitTimeoutError:
            print("Timeout: No speech detected.")
            return None
        except sr.UnknownValueError:
            print("Sorry, I couldn't understand the audio.")
            return None
        except sr.RequestError as e:
            print(f"Error while requesting the speech recognition service: {e}")
            return None

# Define text-to-speech function
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    os.system("response.mp3")

# Define chat function
def chat():
    print("Bot: Hello! How can I assist you today?")
    while True:
        inp = listen_to_voice()
        if inp and inp.lower() == "quit":
            break

        if inp:
            response = generate_response_from_model(inp, model, words, labels, data)
            print("Bot:", response)
            text_to_speech(response)

# Start the chatbot
if __name__ == "__main__":
    chat()


FileNotFoundError: [Errno 2] No such file or directory: 'training_data.pkl'